In [ ]:
## Install required libs
#!pip install tensorflow==1.13.2
!pip uninstall tensorflow
!pip install tensorflow-gpu==1.13.2
!pip install keras-applications==1.0.7
!pip install keras==2.2.4
!pip install image-classifiers==1.0.*
!pip install efficientnet==1.0.*
!pip install h5py==2.10.0

In [ ]:
### please update Albumentations to version>=0.3.0 for `Lambda` transform support
!pip install -U albumentations>=0.3.0 --user 
!pip install -U segmentation-models --user
!pip install -q -U segmentation-models-pytorch --user 

In [ ]:
!pip install tensorflow==1.13.2

In [ ]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
import tensorflow as tf
tf.test.is_gpu_available() # True/False

# Or only check for gpu's with cuda support
tf.test.is_gpu_available(cuda_only=True) 

In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import argparse
import matplotlib.pyplot as plt
import albumentations
import segmentation_models_pytorch as smp
import torch
from tqdm import tqdm as tqdm
import sys
from torch.utils.data import DataLoader
import math
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import multiprocessing as mlt
import glob

In [ ]:
#////////////////////////UTILITY FUNCTIONS \\\\\\\\\\\\\\\\\\\\\\\\

# helper function for data visualization
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        #print(image.shape)
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()
    
# helper function for data visualization    
def denormalize(x):
    """Scale image to range 0..1 for correct plot"""
    x_max = np.percentile(x, 98)
    x_min = np.percentile(x, 2)    
    x = (x - x_min) / (x_max - x_min)
    x = x.clip(0, 1)
    return x
    
# helper function for creating RGB of ith mask
def MasktoRGB(mask,color):
    r = np.expand_dims((mask*color[0]),axis=-1)
    g = np.expand_dims((mask*color[1]),axis=-1)
    b = np.expand_dims((mask*color[2]),axis=-1)
    img = np.concatenate((r,g,b), axis=-1)
    return img.astype(int)

def round_clip_0_1(x, **kwargs):
    return x.round().clip(0, 1)

In [ ]:
#////////////////////////DATASET FUNCTION FOR DATALOADER (torch.utils)\\\\\\\\\\\\\\\\\\\\\\\\
class Dataset:
    CLASSES = ['built-up','background']
    colors = [
        (164, 113, 88), # Built-up
        (255, 255, 255) # background
        ]
    mode_choices = ['multiclass', 'multilabel']
    def __init__(
            self, 
            images_dir, 
            masks_dir,
            contr_dir,
            augmentation=None, 
            preprocessing=None,
            mode = 'multiclass'
    ):
        assert (mode in self.mode_choices), "mode type not found. Choose one of these: {}".format(self.mode_choices)
        self.image_paths = [os.path.join(images_dir, image_id) for image_id in np.sort(os.listdir(images_dir))]
        self.mask_paths = [os.path.join(masks_dir, mask_id) for mask_id in np.sort(os.listdir(masks_dir))]
        self.contr_paths = [os.path.join(contr_dir, contr_id) for contr_id in np.sort(os.listdir(contr_dir))]
        self.augmentation = augmentation
        self.preprocessing = preprocessing
        self.mode = mode
    
    def __getitem__(self, i):
        # read data
        image = cv2.cvtColor(cv2.imread(self.image_paths[i]), cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.mask_paths[i])
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
        contr = cv2.imread(self.contr_paths[i])
        contr = cv2.cvtColor(contr, cv2.COLOR_BGR2RGB)

        # convert RGB mask to index 
        one_hot_map = []
        contr_map = np.all(np.equal(contr, self.colors[0]), axis=-1)
        one_hot_map.append(contr_map)
        mask_map = np.all(np.equal(mask, self.colors[0]), axis=-1)
        one_hot_map.append(mask_map)
        back_map = np.all(np.equal(mask, self.colors[1]), axis=-1)
        one_hot_map.append(back_map)
        
        one_hot_map = np.stack(one_hot_map, axis=-1)
        one_hot_map = one_hot_map.astype('float32')
        
        labels = np.argmax(one_hot_map, axis=-1)

        if self.mode == self.mode_choices[1]:
            # extract certain classes from mask (e.g. cars)
            masks = [(labels == v) for v in range(3)]
            mask = np.stack(masks, axis=-1).astype('float')
        else:
            mask = labels

        #print(mask.shape)

        # apply augmentations
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']

        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']

        image = np.transpose(image, (2, 0, 1)).astype('float32')
        if self.mode == self.mode_choices[1]:
            mask = np.transpose(mask, (2, 0, 1)).astype('int64')

        return image,mask.astype('int64')#,self.image_ids[i]
        
    def __len__(self):
        return len(self.image_paths)

In [ ]:
#////////////////////////Augmentations\\\\\\\\\\\\\\\\\\\\\\\\
def get_training_augmentation():
    train_transform = [albumentations.HorizontalFlip(p=0.5),
        albumentations.ShiftScaleRotate(scale_limit=0.5, rotate_limit=0, shift_limit=0.1, p=1, border_mode=0),
        albumentations.PadIfNeeded(min_height=320, min_width=320, always_apply=True, border_mode=0),
        albumentations.RandomCrop(height=320, width=320, always_apply=True),
        albumentations.IAAAdditiveGaussianNoise(p=0.2),
        #albumentations.IAAPerspective(p=0.5),
        albumentations.OneOf(
            [
                albumentations.CLAHE(p=1),
                albumentations.RandomBrightness(p=1),
                albumentations.RandomGamma(p=1),
            ],
            p=0.9,
        ),
        albumentations.OneOf(
            [
                albumentations.IAASharpen(p=1),
                albumentations.Blur(blur_limit=3, p=1),
                albumentations.MotionBlur(blur_limit=3, p=1),
            ],
            p=0.9,
        ),
        albumentations.OneOf(
            [
                albumentations.RandomContrast(p=1),
                albumentations.HueSaturationValue(p=1),
            ],
            p=0.9,
        ),
        albumentations.Lambda(mask=round_clip_0_1)
    ]
    
    return albumentations.Compose(train_transform)

def get_validation_augmentation():
    """Add paddings to make image shape divisible by 32"""
    test_transform = [
        albumentations.PadIfNeeded(384, 480)
    ]
    return albumentations.Compose(test_transform)

In [ ]:
#////////////////////////Pre Processing\\\\\\\\\\\\\\\\\\\\\\\\
def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform

    Args:
        preprocessing_fn (callbale): data normalization function
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose

    """

    _transform = [
        albumentations.Lambda(image=preprocessing_fn),
    ]
    return albumentations.Compose(_transform)

In [ ]:
#//////////////////////// Setting the parameters \\\\\\\\\\\\\\\\\\\\\\\\
data_dir = "D://LUMS_RA//Seg_data" # Path to Dataset
output_dir = "D://LUMS_RA//Models//Segmentation//trained_model//DeepLabV3+" # Path to save model
n_classes = 3
BACKBONE = "efficientnetb3"
BATCH_SIZE_TRAIN = 32
BATCH_SIZE_VAL = 16
LR = 0.00004
EPOCHS = 40
mode = 'multiclass'
mode_choices = ['multiclass', 'multilabel'] #See documentation for more details.
threshold = 0.5 #Threshold for computation of tp,fp,fn,tn

assert (mode in mode_choices), "Mode can be only from one of these: {}".format(mode_choices)
#Define Network/ Models and it's parameters. See documentation for more details.
ENCODER = 'resnet50'
ENCODER_WEIGHTS = 'imagenet'
ACTIVATION = 'sigmoid' # could be None for logits or 'softmax2d' for multiclass segmentation
model = smp.DeepLabV3Plus(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=n_classes, 
    activation=ACTIVATION
    )

#Pre-processing function to pre-process whole data for encoder
preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

# define optimizer
optimizer = torch.optim.Adam([ 
    dict(params=model.parameters(), lr=LR),
])

'''
Define Loss function. See smp.losses for more details:
Types required are as follows:
Ground Truth : torch.tensor of shape(N,1,H,W) for binary mode, (N,H,W) for multiclass mode and (N,C,H,W) 
               for multilabel mode.
Prediction : (N,1,H,W) for binary mode, (N,C,H,W) for multiclass mode and multilabel mode.

To see details of mode, see documentation of smp library.
'''
dice_loss = smp.losses.DiceLoss(mode='multiclass')
loss = dice_loss

#Define metrics to compute
metrics = {"IOU Score": smp.metrics.iou_score,
           "F1 Score": smp.metrics.f1_score,
           "F_beta score": smp.metrics.fbeta_score,
           "Accuracy": smp.metrics.accuracy,
           "Recall": smp.metrics.recall}

# Set device: `cuda` or `cpu`
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

In [ ]:
#//////////////////////// Visualize the data \\\\\\\\\\\\\\\\\\\\\\\\
imgs_dir = os.path.join(data_dir, 'images')
segm_dir = os.path.join(data_dir, 'segmentations')
contr_dir = os.path.join(data_dir, 'contours')

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataset = Dataset(
    imgs_dir, 
    segm_dir,
    contr_dir,
    preprocessing=get_preprocessing(preprocessing_fn),
)

image, mask = dataset[1]
print(mask.dtype)
#print(image.shape)
#print(mask.shape)
plt.imshow(mask[0]+2*mask[1]+3*mask[2])
plt.show()

In [ ]:
#//////////////////////// Training \\\\\\\\\\\\\\\\\\\\\\\\
ct_train = False
overwrite = False

images_train_dir = os.path.join(data_dir, 'images')
buildings_train_dir = os.path.join(data_dir, 'segmentations')
boundaries_train_dir = os.path.join(data_dir, 'contours')

images_test_dir = os.path.join(data_dir, 'test_images')
buildings_test_dir = os.path.join(data_dir, 'test_segmentations')
boundaries_test_dir = os.path.join(data_dir, 'test_contours')

if ct_train:
    assert os.path.exists(output_dir), "Model not found at path: {}".format(output_dir)
    model = torch.load(os.path.join(output_dir,'best_model.h5'), map_location=DEVICE)
    print('Loaded pre-trained DeepLabV3+ model!')
else:
    assert os.path.exists(output_dir) && overwrite, "Path already exists, cannot over write. Please set the overwrite flag to high, if you want to overwrite"
    if not :
        os.makedirs(output_dir)
    else:
        print("Warining Model path {} already exists, overwriting it".format(output_dir))
    

# Dataset for train images
train_dataset = Dataset(
    images_train_dir,
    buildings_train_dir,
    boundaries_train_dir,
    augmentation=get_training_augmentation(),
    preprocessing=get_preprocessing(preprocessing_fn),
)

# Dataset for validation images
valid_dataset = Dataset(
    images_test_dir,
    buildings_test_dir,
    boundaries_test_dir,
    augmentation=get_validation_augmentation(),
    preprocessing=get_preprocessing(preprocessing_fn),
)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE_TRAIN, shuffle=True, num_workers=8)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE_VAL, shuffle=False, num_workers=4)

best_iou_score = 0.0
print("Total Epochs: {}".format(EPOCHS))
for i in range(0, EPOCHS):
    # Perform training & validation
    print('\nEpoch: {}'.format(i))
    model.train()
    train_logs = {}
    loss_meter = np.array([])
    metrics_meters = {name : np.array([]) for name, func in metrics.items()}
    verbose = True

    with open(os.path.join(output_dir, 'train_logs.txt'), "w") as f:
        f.write("Epoch #: Logs" + "\n")

    with open(os.path.join(output_dir, 'valid_logs.txt'), "w") as f:
        f.write("Epoch #: Logs" + "\n")

    with tqdm(train_loader, desc="Train", file=sys.stdout, disable=not verbose) as iterator:
        for x, y in iterator:
            x, y = x.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            y_pred = model.forward(x)
            loss_Val = loss(y_pred, y)
            loss_Val.backward()
            optimizer.step()
            # update loss logs
            loss_value = loss_Val.cpu().detach().numpy()
            loss_meter = np.append(loss_meter, loss_value)
            loss_logs = {"DICE_LOSS": np.mean(loss_meter)}
            train_logs.update(loss_logs)

            if mode == mode_choices[1]:
                tp, fp, fn, tn = smp.metrics.get_stats(torch.argmax(y_pred,dim=1), y, mode=mode,
                                                       threshold=threshold)
            else:
                tp, fp, fn, tn = smp.metrics.get_stats(torch.argmax(y_pred, dim=1), y, mode=mode,
                                                       num_classes=n_classes)

            # update metrics logs
            for name, metric_fn in metrics.items():
                metric_value = metric_fn(tp, fp, fn, tn).cpu().detach().numpy()
                metrics_meter = metrics_meters[name]
                metrics_meters[name] = np.append(metrics_meter, metric_value)

            metrics_logs = {name: np.mean(values) for name, values in metrics_meters.items()}
            train_logs.update(metrics_logs)

            if verbose:
                str_logs = ['{} - {:.4}'.format(k, v) for k, v in train_logs.items()]
                s = ', '.join(str_logs)
                iterator.set_postfix_str(s)

    torch.save(model, os.path.join(output_dir, 'best_model.h5'))

    with open(os.path.join(output_dir, 'train_logs.txt'), 'a') as f:
        f.write("{} : {}".format(i, str(train_logs)) + "\n")

    model.eval()
    valid_logs = {}
    loss_meter = np.array([])
    metrics_meters = {name : np.array([]) for name, func in metrics.items()}
    verbose = True

    with tqdm(valid_loader, desc="Valid", file=sys.stdout, disable=not verbose) as iterator:
        for x, y in iterator:
            x, y = x.to(DEVICE), y.to(DEVICE)
            with torch.no_grad():
                y_pred = model.forward(x)
                loss_Val = loss(y_pred, y)

            # update loss logs
            loss_value = loss_Val.cpu().detach().numpy()
            loss_meter = np.append(loss_meter, loss_value)
            loss_logs = {"DICE_LOSS": np.mean(loss_meter)}
            valid_logs.update(loss_logs)

            if mode == mode_choices[1]:
                tp, fp, fn, tn = smp.metrics.get_stats(torch.argmax(y_pred, dim=1), y, mode=mode,
                                                       threshold=threshold)
            else:
                tp, fp, fn, tn = smp.metrics.get_stats(torch.argmax(y_pred, dim=1), y, mode=mode,
                                                       num_classes=n_classes)


            # update metrics logs
            for name,metric_fn in metrics.items():
                metric_value = metric_fn(tp, fp, fn, tn).cpu().detach().numpy()
                metrics_meter = metrics_meters[name]
                metrics_meters[name] = np.append(metrics_meter, metric_value)

            metrics_logs = {name: np.mean(vals) for name, vals in metrics_meters.items()}
            valid_logs.update(metrics_logs)

            if verbose:
                str_logs = ['{} - {:.4}'.format(k, v) for k, v in valid_logs.items()]
                s = ', '.join(str_logs)
                iterator.set_postfix_str(s)

    with open(os.path.join(output_dir, 'valid_logs.txt'), 'a') as f:
        f.write("{} : {}".format(i, str(valid_logs)) + "\n")

In [ ]:
#//////////////////////// Visualize Results on an image from Test Set \\\\\\\\\\\\\\\\\\\\\\\\
images_dir = os.path.join(args.data_dir, 'test_images')
masks_dir = os.path.join(args.data_dir, 'test_segmentations')
contour_dir = os.path.join(args.data_dir, 'test_contours')

# Set device: `cuda` or `cpu`   
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
test_dataset = Dataset(
    images_dir,
    masks_dir,
    contour_dir,
    preprocessing=get_preprocessing(preprocessing_fn),
)

# load best weights
best_model = torch.load(os.path.join(args.output_dir,'best_model.h5'), map_location=DEVICE)

image, gt_mask = test_dataset[0]
#gt_mask = np.transpose(gt_mask,(1,2,0))

x_tensor = torch.from_numpy(image).to(DEVICE).unsqueeze(0)
#image = image.squeeze()
image = np.transpose(image,(1,2,0))

pred_mask = best_model(x_tensor)
pr_mask = pred_mask.squeeze()
pr_mask = pr_mask.detach().squeeze().cpu().numpy()
pr_mask = np.argmax(pr_mask, axis=0)

#print(pr_mask.shape)
#pr_mask = np.transpose(pr_mask,(1,2,0))
'''
print(pr_mask.shape)
print(gt_mask.shape)
print(image.shape)'''
gt_img = 3*gt_mask[0]+2*gt_mask[1]+gt_mask[2]
pr_img = 3*(pr_mask == 0) + 2*(pr_mask == 1) + (pr_mask == 2)
print(pr_img.shape)

#print(name)
visualize(
    image=denormalize(image.squeeze()),
    gt_mask=gt_img.astype(np.uint8),
    pr_mask=pr_img.astype(np.uint8),
)

In [ ]:
#//////////////////////// Evaluate on Images \\\\\\\\\\\\\\\\\\\\\\\\
imgs_dir = os.path.join(data_dir, 'images') # Input the path to images
pred_dir = os.path.join(data_dir, 'predictions') # The path where outputs are saved
if not os.path.exists(pred_dir):
    os.makedirs(pred_dir)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# load best weights
model = torch.load(os.path.join(output_dir,'best_model.h5'), map_location=DEVICE)
dataset = Dataset(
    imgs_dir, 
    imgs_dir, 
    classes=CLASSES, 
    preprocessing=get_preprocessing(preprocessing_fn),
)
i = 0
with tqdm(dataset, file=sys.stdout) as iterator:
    for it in iterator:
        image, _,f_name = dataset[i]
        x_tensor = torch.from_numpy(image).to(DEVICE).unsqueeze(0)
        image = image.squeeze()
        image = np.transpose(image,(1,2,0))
        pred_mask = model(x_tensor)
        pr_mask = pred_mask.round().squeeze()
        pr_mask = pr_mask.detach().squeeze().cpu().numpy()

        if mode == smp.losses.constants.BINARY_MODE:
            pr_mask = np.expand_dims(pr_mask, 0)

        #print(pr_mask.shape)
        pr_mask = np.transpose(pr_mask,(1,2,0))
        pr_img = np.zeros((pr_mask.shape[0],pr_mask.shape[1],3))
        for j in range(pr_mask.shape[2]):
            pr_img = pr_img+ MasktoRGB(pr_mask[...,j],Dataset.colors[j])

        #f_name = "{}.png".format(i+1)
        save_path = os.path.join(pred_dir,f_name)
        plt.imsave(save_path,pr_img.astype(np.uint8))
        i = i+1
        #print("Output for Image: {} has been saved!".format(f_name))

In [ ]:
#//////////////////////// Evaluate on Tiff Files \\\\\\\\\\\\\\\\\\\\\\\\
input_imgs = os.path.join(data_dir, 'tiff_images') # Input the path to tiff images

target_size = (256, 256)
padding_pixels = 32
padding_value = 0

if not os.path.exists(args.output_dir):
        os.makedirs(args.output_dir)

model = torch.load(os.path.join(output_dir, 'best_model.h5'), map_location=DEVICE)
imgs = [file for file in glob.glob(input_imgs) if file.endswith('.tif')]

assert len(imgs) > 0, "The number of images equal to zero"

num_processes = 1
print("Running on {} images using {} parallel processes".format(len(imgs), num_processes))

pos = len(input_imgs.split('*')[0]) #Related to naming of output file

args = [[img, DEVICE, model, args.output_dir, pos] for img in imgs]

if num_processes > 1:
    p = mlt.Pool(num_processes)
    (p.map(process, args))
    p.close()
else:
    for arg in args:
        process(arg)
print("Completed")
def process(args):
    img_path, device, model, output_dir, pos = args

    file_name = os.path.split(img_path)[-1].split('.')[0]

    sub_dirs = os.path.split(img_path[pos:])[0]
    out_path = os.path.join(output_dir,sub_dirs)
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    save_path = os.path.join(out_path, file_name + "_preds.png")
    chk_path = save_path.split('.')[0]+'.tif'
    if os.path.exists(save_path) or os.path.exists(chk_path):
        print("Prediction for file {} already exists, skipping..!".format(file_name))
        return
    img = np.array(Image.open(img_path))
    img = cv2.copyMakeBorder(img, padding_pixels, padding_pixels, padding_pixels, padding_pixels,
                             cv2.BORDER_CONSTANT, value=padding_value)
    src_im_height = img.shape[0]
    src_im_width = img.shape[1]

    cols = (math.ceil(src_im_height/target_size[0]))
    rows = (math.ceil(src_im_width/target_size[1]))

    print("Total {} patches for the given image {}".format(rows*cols, file_name))
    combined_image = np.zeros((cols*target_size[0], rows*target_size[1]), dtype=np.uint8) * 255
    #useful_portion = np.zeros((src_im_height-2*padding_pixels, src_im_width-2*padding_pixels), dtype=np.uint8)

    preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)
    preprocessing = get_preprocessing(preprocessing_fn)

    x1, y1, idx = 0, 0, 0
    while y1 < src_im_height:
        y2 = y1 + target_size[0] + 2 * padding_pixels
        while x1 < src_im_width:
            x2 = x1 + target_size[1] + 2 * padding_pixels
            img_crop = img[y1: y2, x1: x2]
            pad_bottom = y2 - src_im_height if y2 > src_im_height else 0
            pad_right = x2 - src_im_width if x2 > src_im_width else 0

            if pad_bottom > 0 or pad_right > 0:
                img_crop = cv2.copyMakeBorder(img_crop, 0, pad_bottom, 0, pad_right,
                                              cv2.BORDER_CONSTANT, value=padding_value)

            sample = preprocessing(image=img_crop)
            image = sample['image']
            image = np.transpose(image, (2, 0, 1)).astype('float32')
            x_tensor = torch.from_numpy(image).to(device).unsqueeze(0)
            pred_mask = model(x_tensor)
            pr_mask = pred_mask.squeeze()
            pr_mask = pr_mask.detach().squeeze().cpu().numpy()
            mask = pr_mask.round()
            sizex, sizey = target_size
            patch = mask[padding_pixels:sizex + padding_pixels, padding_pixels:sizey + padding_pixels]
            combined_image[y1: y1+target_size[1], x1: x1+target_size[0]] = patch

            x1 += target_size[0]
            idx += 1
            print("Patch {} done for file {}".format(idx, file_name))
            #break
        x1 = 0
        y1 += target_size[1]
        #break

    #useful_portion = combined_image[:src_im_height-2*padding_pixels, :src_im_width-2*padding_pixels]
    plt.imsave(save_path, combined_image[:src_im_height-2*padding_pixels, :src_im_width-2*padding_pixels])

    print_line = "Processing completed for {}..!".format(file_name)
    print(print_line)
